# Simplified AES Plaintext Recovery (FNN)
In this experiment, the network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import SimplifiedAESDatasetCiphertextPlaintext
from pipeline import *

2023-06-05 09:33:52.084685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [3]:
data = SimplifiedAESDatasetCiphertextPlaintext('small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [4]:
train_labels = train_labels[:500]
train_samples = train_samples[:500]

In [5]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (500, 16)
===== Label Shape: (16,)
===== Training Samples Shape: (500, 16)
===== Sample Shape: (16,)
===== Testing Labels Shape: (649947, 16)
===== Testing Samples Shape: (649947, 16)


## Creating the model

In [6]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [7]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])
print("Output dimension: {}".format(dim))

# units per hidden layer
units = dim*16
print("Units per hidden layer: {}".format(units))

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 1000
batch_size = 100

Output dimension: 16
Units per hidden layer: 256


### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [8]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               4352      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 16)                4112      
                                                                 
Total params: 205,840
Trainable params: 205,840
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [9]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/1000
5/5 [==============================] - 1s 55ms/step - loss: 0.2327 - accuracy: 0.1467 - binary_accuracy: 0.6417 - val_loss: 0.1936 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7475
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1800 - accuracy: 0.2111 - binary_accuracy: 0.7415 - val_loss: 0.1694 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7387
Epoch 3/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.1710 - accuracy: 0.2133 - binary_accuracy: 0.7447 - val_loss: 0.1623 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7588
Epoch 4/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1600 - accuracy: 0.1578 - binary_accuracy: 0.7700 - val_loss: 0.1532 - val_accuracy: 0.0200 - val_binary_accuracy: 0.7675
Epoch 5/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1516 - accuracy: 0.0267 - binary_accuracy: 0.7782 - val_loss: 0.1428 - val_accuracy: 0.1200 - val_binary_accuracy: 0.7937
Epoch 6/1000
5/

In [10]:
print(history.history['loss'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

[0.23270000517368317, 0.17999380826950073, 0.1710319072008133, 0.1600344479084015, 0.1516106277704239, 0.13904531300067902, 0.12700146436691284, 0.1163424700498581, 0.10639842599630356, 0.0988161563873291, 0.09070663154125214, 0.08443915843963623, 0.07915252447128296, 0.07398084551095963, 0.0688399001955986, 0.06435774266719818, 0.06125595048069954, 0.05603983253240585, 0.05331557244062424, 0.0493965670466423, 0.04501345381140709, 0.04248121753334999, 0.039410460740327835, 0.03709980472922325, 0.03460447117686272, 0.03263312578201294, 0.030630582943558693, 0.02864670194685459, 0.027184903621673584, 0.025326408445835114, 0.025075552985072136, 0.02425752766430378, 0.021700717508792877, 0.02045292966067791, 0.018865790218114853, 0.0181280504912138, 0.01677386462688446, 0.01558989379554987, 0.014466234482824802, 0.013972287066280842, 0.013039893470704556, 0.012156488373875618, 0.011803263798356056, 0.010862383991479874, 0.010598442517220974, 0.010106401517987251, 0.009482254274189472, 0.00

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [93]:
test_labels2 = []
test_samples2 = []
index = 0

for t in test_labels:
    contains = False
    for tt in train_labels:
        if np.array_equal(t, tt):
            contains = True
            break
    if not contains:
        test_labels2.append(t)
        test_samples2.append(test_samples[index])
    index += 1
    


In [94]:
print(len(test_labels))
print(len(test_labels2))

649947
195679


In [95]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

6500/6500 [==============================] - 11s 2ms/step - loss: 0.0511 - accuracy: 0.3570 - binary_accuracy: 0.9417
Test loss: 0.05113872513175011
Test accuracy: 0.35704296827316284


In [97]:
results = neural_network.evaluate(np.array(test_samples2), np.array(test_labels2), batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

1957/1957 [==============================] - 4s 2ms/step - loss: 0.1604 - accuracy: 0.3218 - binary_accuracy: 0.8174
Test loss: 0.16042689979076385
Test accuracy: 0.3218485414981842


### Plaintext Recovery in action

In [111]:
size = 1000

In [112]:
predictions = [predict_sample(neural_network, test_samples2[i]) for i in range(size)]

1/1 [==============================] - 0s 24ms/step


In [113]:
metrics = [correct_and_metrics((predictions[i], test_labels2[i])) for i in range(size)]

In [114]:
correct_bytes = 0
correct_predictions = 0
for m in metrics:
    correct_bytes += m[0]
    correct_predictions += m[1]
                             
print("Correct bytes: {}".format(correct_bytes))
print("Byte accuracy: {}".format(correct_bytes/(2*size)))
print("Correct predictions: {}".format(correct_predictions))
print("Prediction accuracy: {}".format(correct_predictions/size))

Correct bytes: 431
Byte accuracy: 0.2155
Correct predictions: 77
Prediction accuracy: 0.077


In [115]:
wrong_idx = []
correct_idx = []

for i in range(size):
    correct = prediction_to_string(test_labels2[i])
    predicted = prediction_to_string(predictions[i])
    if correct != predicted:
        wrong_idx.append(i)
    else:
        correct_idx.append(i)
    print("correct-> " + correct + " | " + predicted + " <-actual")

correct-> do | dm <-actual
correct-> no | no <-actual
correct-> 33 | nf <-actual
correct-> :3 | rg <-actual
correct-> o  | n  <-actual
correct-> pa |  e <-actual
correct-> (l | 
q <-actual
correct-> o, | ni <-actual
correct-> e, | cl <-actual
correct-> )  | 9( <-actual
correct-> 
t | Be <-actual
correct-> y  | x` <-actual
correct-> ow | Oe <-actual
correct-> a  | m  <-actual
correct-> pr |  w <-actual
correct-> op | n  <-actual
correct-> ee | ee <-actual
correct-> em | um <-actual
correct-> 
3 | he <-actual
correct-> 4: | v( <-actual
correct-> wo | go <-actual
correct-> rd | ri <-actual
correct-> LO | Go <-actual
correct-> RD | ~` <-actual
correct-> to | dn <-actual
correct-> e, | cl <-actual
correct-> g, | gl <-actual
correct->  3 | 
a <-actual
correct-> 4: | v( <-actual
correct-> So | cj <-actual
correct-> ,
 | ra <-actual
correct-> pr |  w <-actual
correct-> op | n  <-actual
correct-> sy |  <-actual
correct-> ph |  i <-actual
correct-> Is | ip <-actual
correct-> ra | rm <-actual
cor